In [25]:
!pip install llama-index



import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import nest_asyncio
nest_asyncio.apply()


In [26]:
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    #model="llama3.2:latest",
    model="mistral:7b",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [27]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Response
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import PairwiseComparisonEvaluator
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd

pd.set_option("display.max_colwidth", 0)

In [28]:
evaluator_pairwise = PairwiseComparisonEvaluator(llm=ollama_llm)

In [29]:
documents = SimpleDirectoryReader("./test_wiki_data/").load_data()

In [30]:
# create vector index
splitter_512 = SentenceSplitter(chunk_size=512,chunk_overlap=64)
vector_index1 = VectorStoreIndex.from_documents(
    documents, transformations=[splitter_512]
)

splitter_128 = SentenceSplitter(chunk_size=128,chunk_overlap=64)
vector_index2 = VectorStoreIndex.from_documents(
    documents, transformations=[splitter_128]
)

In [31]:
query_engine1 = vector_index1.as_query_engine(similarity_top_k=2)
query_engine2 = vector_index2.as_query_engine(similarity_top_k=8)

In [32]:
# define jupyter display function
def display_eval_df(query, response1, response2, eval_result) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Reference Response (Answer 1)": response2,
            "Current Response (Answer 2)": response1,
            "Score": eval_result.score,
            "Reason": eval_result.feedback,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "300px",
            "overflow-wrap": "break-word",
        },
        subset=["Current Response (Answer 2)", "Reference Response (Answer 1)"]
    )
    display(eval_df)

In [33]:
# query_str = "How did New York City get its name?"
query_str = "What was the role of NYC during the American Revolution?"
# query_str = "Tell me about the arts and culture of NYC"
response1 = str(query_engine1.query(query_str))
response2 = str(query_engine2.query(query_str))

In [34]:
eval_result = await evaluator_pairwise.aevaluate(
    query_str, response=response1, second_response=response2
)

In [35]:
display_eval_df(query_str, response1, response2, eval_result)

,Query,Reference Response (Answer 1),Current Response (Answer 2),Score,Reason
0,What was the role of NYC during the American Revolution?,"During the American Revolution, New York City served as a significant location for several key events. It was initially a base for British troops and a haven for Loyalist refugees and escaped slaves. The Battle of Long Island, one of the largest battles of the war, took place within the modern-day borough of Brooklyn in August 1776. After the battle, the city became the military and political base of operations for the British in North America. The only attempt at a peaceful resolution to the war occurred at the Conference House on Staten Island between American delegates and British general Lord Howe in September 1776. The city was also the national capital from 1785 until 1790, hosting several events of national scope during this period, such as the inauguration of the first President of the United States, George Washington, and the assembly of the first United States Congress and Supreme Court. After the war, New York City experienced extensive immigration from various regions, including German provinces, which significantly influenced the city's population and politics. The city was also under British control for a brief period before being regained by the Dutch and then permanently named New York.","During the American Revolution, New York City served as a military and political base of operations for the British forces in North America. It was also a haven for Loyalist refugees and escaped slaves who joined the British lines for freedom newly promised by the Crown for all fighters. The Battle of Long Island, the largest battle of the American Revolutionary War, was fought within the modern-day borough of Brooklyn. After the British forces evacuated at the close of the war in 1783, they transported 3,000 freedmen for resettlement in Nova Scotia. The city also hosted several events of national scope during this period, such as the inauguration of the first President of the United States, George Washington, and the assembly of the first United States Congress and the Supreme Court of the United States.",0.500000,


In [36]:
evaluator_pairwise_nc = PairwiseComparisonEvaluator(
    llm=ollama_llm, enforce_consensus=False
)

In [37]:
eval_result = await evaluator_pairwise_nc.aevaluate(
    query_str, response=response1, second_response=response2
)

In [38]:
display_eval_df(query_str, response1, response2, eval_result)

,Query,Reference Response (Answer 1),Current Response (Answer 2),Score,Reason
0,What was the role of NYC during the American Revolution?,"During the American Revolution, New York City served as a significant location for several key events. It was initially a base for British troops and a haven for Loyalist refugees and escaped slaves. The Battle of Long Island, one of the largest battles of the war, took place within the modern-day borough of Brooklyn in August 1776. After the battle, the city became the military and political base of operations for the British in North America. The only attempt at a peaceful resolution to the war occurred at the Conference House on Staten Island between American delegates and British general Lord Howe in September 1776. The city was also the national capital from 1785 until 1790, hosting several events of national scope during this period, such as the inauguration of the first President of the United States, George Washington, and the assembly of the first United States Congress and Supreme Court. After the war, New York City experienced extensive immigration from various regions, including German provinces, which significantly influenced the city's population and politics. The city was also under British control for a brief period before being regained by the Dutch and then permanently named New York.","During the American Revolution, New York City served as a military and political base of operations for the British forces in North America. It was also a haven for Loyalist refugees and escaped slaves who joined the British lines for freedom newly promised by the Crown for all fighters. The Battle of Long Island, the largest battle of the American Revolutionary War, was fought within the modern-day borough of Brooklyn. After the British forces evacuated at the close of the war in 1783, they transported 3,000 freedmen for resettlement in Nova Scotia. The city also hosted several events of national scope during this period, such as the inauguration of the first President of the United States, George Washington, and the assembly of the first United States Congress and the Supreme Court of the United States.",1.000000,"In this case, both assistants provide accurate and relevant information about the role of NYC during the American Revolution. However, Assistant A's response is more comprehensive in terms of detailing the city's role as a military base for the British forces, a haven for Loyalist refugees and escaped slaves, and its significance in hosting key events such as the inauguration of the first President of the United States and the assembly of the first US Congress and Supreme Court. Assistant B also mentions these points but adds additional information about the Conference House on Staten Island and the city's immigration history, which makes their response slightly more detailed. However, the factor that tips the balance in favor of Assistant A is the clarity with which they present the information. While both assistants are concise, Assistant A presents the information in a way that is easier to understand and follow for the user. For instance, they clearly state that New York City served as a military and political base of operations for the British after the Battle of Long Island, whereas Assistant B mentions this fact but does not explicitly state that it occurred after the battle. In conclusion, while both responses are good, Assistant A provides a clearer and more comprehensive answer to the user's question, making them the better assistant in this case. [[A]]"


In [39]:
eval_result = await evaluator_pairwise_nc.aevaluate(
    query_str, response=response2, second_response=response1
)

In [40]:
display_eval_df(query_str, response2, response1, eval_result)

,Query,Reference Response (Answer 1),Current Response (Answer 2),Score,Reason
0,What was the role of NYC during the American Revolution?,"During the American Revolution, New York City served as a military and political base of operations for the British forces in North America. It was also a haven for Loyalist refugees and escaped slaves who joined the British lines for freedom newly promised by the Crown for all fighters. The Battle of Long Island, the largest battle of the American Revolutionary War, was fought within the modern-day borough of Brooklyn. After the British forces evacuated at the close of the war in 1783, they transported 3,000 freedmen for resettlement in Nova Scotia. The city also hosted several events of national scope during this period, such as the inauguration of the first President of the United States, George Washington, and the assembly of the first United States Congress and the Supreme Court of the United States.","During the American Revolution, New York City served as a significant location for several key events. It was initially a base for British troops and a haven for Loyalist refugees and escaped slaves. The Battle of Long Island, one of the largest battles of the war, took place within the modern-day borough of Brooklyn in August 1776. After the battle, the city became the military and political base of operations for the British in North America. The only attempt at a peaceful resolution to the war occurred at the Conference House on Staten Island between American delegates and British general Lord Howe in September 1776. The city was also the national capital from 1785 until 1790, hosting several events of national scope during this period, such as the inauguration of the first President of the United States, George Washington, and the assembly of the first United States Congress and Supreme Court. After the war, New York City experienced extensive immigration from various regions, including German provinces, which significantly influenced the city's population and politics. The city was also under British control for a brief period before being regained by the Dutch and then permanently named New York.",1.000000,"In this comparison, both AI assistants have provided accurate, relevant, and helpful responses to the user's question about the role of NYC during the American Revolution. Both have covered essential points such as the city serving as a base for British troops, hosting key events like the Battle of Long Island, and being the temporary national capital. They also mention the presence of Loyalist refugees and escaped slaves in the city. However, Assistant A provides slightly more detail about the Conference House on Staten Island and the immigration patterns that influenced NYC's population and politics after the war. Additionally, Assistant A mentions that the city was under British control for a brief period before being regained by the Dutch and then permanently named New York, which is not mentioned in Assistant B's response. Given these differences, I would evaluate Assistant A as providing a slightly more detailed and comprehensive answer compared to Assistant B. However, both responses are of high quality, and either could be considered acceptable depending on the specific requirements or preferences of the user. [[A]]"


In [41]:
query_str = "Tell me about the arts and culture of NYC"
response1 = str(query_engine1.query(query_str))
response2 = str(query_engine2.query(query_str))

In [42]:
eval_result = await evaluator_pairwise.aevaluate(
    query_str, response=response1, second_response=response2
)

In [43]:
display_eval_df(query_str, response1, response2, eval_result)

,Query,Reference Response (Answer 1),Current Response (Answer 2),Score,Reason
0,Tell me about the arts and culture of NYC,"New York City is renowned for its vibrant arts and cultural scene. It boasts over 2,000 arts and cultural organizations and more than 500 art galleries. The city government funds the arts with a larger annual budget than the National Endowment for the Arts. Many significant cultural movements originated in New York City, such as the Harlem Renaissance, abstract expressionism, and hip-hop. The city has been instrumental in the development of various forms of music, including jazz, punk, hardcore, salsa, freestyle, Tin Pan Alley, certain forms of jazz, and (along with Philadelphia) disco. New York City is also home to a flourishing scene for Jewish American literature. Some of its cultural institutions, built by wealthy business magnates in the 19th century, have become internationally renowned, such as Carnegie Hall and the Metropolitan Museum of Art. In contemporary life, New York City continues to be described as the cultural capital of the world. It has a thriving entertainment industry, with many films, television series, books, and other media being set there. The city is also known for its diversity, which is celebrated in its various districts and monuments that are major landmarks, including three of the world's ten most visited tourist attractions.","New York City has a rich and diverse cultural scene, often described as the cultural capital of the world. It has been instrumental in the birth and development of numerous major American cultural movements such as the Harlem Renaissance, abstract expressionism, hip-hop, punk, salsa, freestyle, Tin Pan Alley, certain forms of jazz, and disco. The city is also known for its dance culture, with New York Fashion Week being one of the world's preeminent fashion events. The city has a flourishing scene for Jewish American literature and has been the setting for numerous novels, movies, and television programs. Many of its landmarks, skyscrapers, and parks are known around the world. The Empire State Building is a global standard of reference to describe the height and length of other structures. In terms of contemporary life, New York City in the 21st century has emerged as a global node of creativity and entrepreneurship, with institutions like Columbia University, New York University, and the City University of New York system contributing significantly to this. The city is also known for its fast pace, which led to the phrase ""New York minute."" The Stonewall Inn in Greenwich Village is considered the historic epicenter of LGBTQ+ culture and the birthplace of the modern gay rights movement.",0.500000,
